In [ ]:
# !pip install langchain langchain_community -q
# #For API Calls
# !pip install huggingface_hub -q
# !pip install transformers -q
# !pip install accelerate -q
# !pip install  bitsandbytes -q
# !pip install langchain-huggingface -q


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
# !pip -q install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 115.8 MB/s eta 0:

# set up env

In [2]:
from dotenv import load_dotenv
import os

# Load .env
load_dotenv()

token = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
if not token:
    raise ValueError("HUGGINGFACEHUB_API_TOKEN is not set!")

# Load data

In [3]:
# !pip install PyPDF2 pypdf -q

In [4]:
from langchain.document_loaders import PyPDFLoader

In [5]:
%pwd

'c:\\d\\generative AI\\Doc2Question\\research'

In [6]:
%cd ..

c:\d\generative AI\Doc2Question


In [7]:
%pwd

'c:\\d\\generative AI\\Doc2Question'

In [8]:
file_path = "data/shrimp_docs.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [9]:
data

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-09-30T15:27:00+08:00', 'author': 'Hongchao Duan, Jun Wang,Yuan Zhang, Xiangyu Wu,Tao Peng,Xuhao Liu and Delong Deng', 'keywords': 'shrimp larvae counting; YOLOv5; regional segmentation; attention mechanism; repeat shrimp removal', 'moddate': '2024-09-30T09:33:33+02:00', 'subject': "Counting shrimp larvae is an essential part of shrimp farming. Due to their tiny size and high density, this task is exceedingly difficult. Thus, we introduce an algorithm for counting densely packed shrimp larvae utilizing an enhanced You Only Look Once version 5 (YOLOv5) model through a regional segmentation approach. First, the C2f and convolutional block attention modules are used to improve the capabilities of YOLOv5 in recognizing the small shrimp. Moreover, employing a regional segmentation technique can decrease the receptive field area, thereby enhancing the shrimp counter's detection performanc

In [10]:
len(data)

19

In [11]:
question_gen = ""
for page in data:
    question_gen += page.page_content

In [12]:
question_gen

'Citation: Duan, H.; Wang, J.; Zhang,\nY.; Wu, X.; Peng, T.; Liu, X.; Deng, D.\nShrimp Larvae Counting Based on\nImproved YOLOv5 Model with\nRegional Segmentation. Sensors 2024,\n24, 6328. https://doi.org/10.3390/\ns24196328\nAcademic Editor: Yongwha Chung\nReceived: 10 September 2024\nRevised: 26 September 2024\nAccepted: 27 September 2024\nPublished: 30 September 2024\nCopyright: © 2024 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed under the terms and\nconditions of the Creative Commons\nAttribution (CC BY) license (https://\ncreativecommons.org/licenses/by/\n4.0/).\nsensors\nArticle\nShrimp Larvae Counting Based on Improved YOLOv5 Model\nwith Regional Segmentation\nHongchao Duan, Jun Wang, Yuan Zhang *, Xiangyu Wu, Tao Peng, Xuhao Liu and Delong Deng\nCentre for Optical and Electromagnetic Research, South China Academy of Advanced Optoelectronics,\nSouth China Normal University, Guangzhou 510006, China; 2021024069@m.scnu.edu

# split data

In [13]:
from langchain.text_splitter import split_text_on_tokens, Tokenizer
from transformers import AutoTokenizer


model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id, max_length=512)

def encode(text):
    return tokenizer.encode(text)

def decode(tokens):
    return tokenizer.decode(tokens, skip_special_tokens=True)



tokenizer_local = Tokenizer(
    tokens_per_chunk=500,
    chunk_overlap=50,
    encode=encode,
    decode=decode
)

chunk_ques_gen = split_text_on_tokens(text=question_gen, tokenizer=tokenizer_local)

c:\d\generative AI\Doc2Question\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (13350 > 512). Running this sequence through the model will result in indexing errors


In [14]:
len(chunk_ques_gen)

30

In [15]:
type(chunk_ques_gen[0])

str

In [16]:
from langchain.docstore.document import Document

In [17]:
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]

In [18]:
document_ques_gen

[Document(metadata={}, page_content='Citation: Duan, H.; Wang, J.; Zhang, Y.; Wu, X.; Peng, T.; Liu, X.; Deng, D. Shrimp Larvae Counting Based on Improved YOLOv5 Model with Regional Segmentation. Sensors 2024, 24, 6328. https://doi.org/10.3390/ s24196328 Academic Editor: Yongwha Chung Received: 10 September 2024 Revised: 26 September 2024 Accepted: 27 September 2024 Published: 30 September 2024 Copyright:  2024 by the authors. Licensee MDPI, Basel, Switzerland. This article is an open access article distributed under the terms and conditions of the Creative Commons Attribution (CC BY) license (https:// creativecommons.org/licenses/by/ 4.0/). sensors Article Shrimp Larvae Counting Based on Improved YOLOv5 Model with Regional Segmentation Hongchao Duan, Jun Wang, Yuan Zhang *, Xiangyu Wu, Tao Peng, Xuhao Liu and Delong Deng Centre for Optical and Electromagnetic Research, South China Academy of Advanced Optoelectronics, South China Normal University, Guangzhou 510006, China; 2021024069@m

In [19]:
type(document_ques_gen[0])

langchain_core.documents.base.Document

# create ques

In [20]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto')
pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=450)
llm_ques_gen_pipeline = HuggingFacePipeline(pipeline=pipeline)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\d\generative AI\Doc2Question\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hu

In [21]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [22]:
from langchain.prompts import PromptTemplate

In [23]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [24]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)


In [25]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [26]:
from langchain.chains.summarize import load_summarize_chain

In [27]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline,
                                          chain_type = "refine",
                                          verbose = True,
                                          question_prompt=PROMPT_QUESTIONS,
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [28]:
len(document_ques_gen)

30

In [29]:
# ques = ques_gen_chain.run(document_ques_gen)

results = []
for chunk in document_ques_gen:
    ques = ques_gen_chain.run([chunk])
    results.append(ques)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16168\105993891.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ques = ques_gen_chain.run([chunk])
Token indices sequence length is longer than the specified maximum sequence length for this model (587 > 512). Running this sequence through the model will result in indexing errors




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
Citation: Duan, H.; Wang, J.; Zhang, Y.; Wu, X.; Peng, T.; Liu, X.; Deng, D. Shrimp Larvae Counting Based on Improved YOLOv5 Model with Regional Segmentation. Sensors 2024, 24, 6328. https://doi.org/10.3390/ s24196328 Academic Editor: Yongwha Chung Received: 10 September 2024 Revised: 26 September 2024 Accepted: 27 September 2024 Published: 30 September 2024 Copyright:  2024 by the authors. Licensee MDPI, Basel, Switzerland. This article is an open access article distributed under the terms and conditions of the Creative Commons Attribution (CC BY) license (https:// creativecommons.org/licenses/by/ 4.0/). sensors Article Shrimp Larvae Counting Based on

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



> Finished chain.

> Finished chain.


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
not accurately identify local regions, while the local image allowed a more precise extraction of features from the shrimp larvae.Sensors 2024, 24, 6328 7 of 19 Figure 5. Image segmentation. (a) Original shrimp larvae image. (b) Segmented image. Figure 6. The feature map referring to the shrimp larvae image. 3.3. The Improved YOLOv5 The core idea of YOLOv5 is to convert the target detection problem into a regression problem, predicting the boundary boundaries and categories of the target in the image. It has high accuracy and a fast inference ability, making it one of the best-performing target detection models ava

In [30]:
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings


In [34]:
# embeddings = OpenAIEmbeddings()
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\d\generative AI\Doc2Question\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling bac

In [35]:

from langchain.vectorstores import Chroma
persist_directory = 'db'


vectordb = Chroma.from_documents(documents=document_ques_gen,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

In [36]:
vectordb.persist()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16168\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [37]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto')
pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=450)
llm_answer_gen = HuggingFacePipeline(pipeline=pipeline)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0


In [38]:
ques_list = ques.split("\n")

In [39]:
ques_list = results
ques_list

['What is the name of the algorithm for counting densely packed shrimp larvae utilizing an enhanced You Only Look Once version 5?',
 'What is the name of the algorithm for counting densely packed shrimp larvae utilizing an enhanced You Only Look Once version 5 (YOLOv5) model through a regional segmentation approach?',
 'How are photoelectric devices for counting shrimp larvae affected by various environmental factors?',
 'What is the name of the deep learning method that Armalivia et al. used to count shrimp larvae in a circular area with a diameter of 40 cm?',
 'What are the three states of shrimp larvae?',
 'What are two widely used techniques for decreasing the size of an image?',
 'YOLO pioneered an innovative approach by utilizing convolutional neural networks on a complete image to predict object classes and bounding boxes via direct feature regression. YOLO pioneered an innovative approach by utilizing convolutional neural networks on a complete image to predict object classes a

In [40]:
from langchain.chains import RetrievalQA

In [41]:
retriever = vectordb.as_retriever()
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen,
                                               chain_type="stuff",
                                               retriever=retriever)


In [42]:
# Answer each question and save to a file
for question in ques_list[:4]:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")




Token indices sequence length is longer than the specified maximum sequence length for this model (2073 > 512). Running this sequence through the model will result in indexing errors


Question:  What is the name of the algorithm for counting densely packed shrimp larvae utilizing an enhanced You Only Look Once version 5?
Answer:  YOLOv5
--------------------------------------------------\n\n
Question:  What is the name of the algorithm for counting densely packed shrimp larvae utilizing an enhanced You Only Look Once version 5 (YOLOv5) model through a regional segmentation approach?
Answer:  an algorithm for counting densely packed shrimp larvae utilizing an enhanced You Only Look Once version 5 (YOLOv5) model through a regional segmentation approach
--------------------------------------------------\n\n
Question:  How are photoelectric devices for counting shrimp larvae affected by various environmental factors?
Answer:  These counters can be affected by various environmental factors, such as lighting conditions and water quality, resulting in considerable counting errors. Furthermore, photoelectric devices are generally effective for shrimp larvae of certain sizes,